In [2]:
import pandas as pd
import numpy as np
from PIL import Image
from tensorflow.keras.applications.densenet import DenseNet121,preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add 
import cv2
import tensorflow as tf
from nltk.tokenize import word_tokenize
import nltk
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
densenet = DenseNet121(weights='/content/drive/MyDrive/Project Data/brucechou1983_CheXNet_Keras_0.3.0_weights.h5',classes = 14,input_shape=(224,224,3))
CNN1 = Model(inputs=[densenet.input], outputs=[densenet.layers[-2].output])
CNN1.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                                              

In [5]:
plot_model(CNN1,show_shapes = True)


Output hidden; open in https://colab.research.google.com to view.

In [6]:
img = cv2.imread("/content/drive/MyDrive/Project Data/NLMCXR_png/CXR1000_IM-0003-1001.png", cv2.IMREAD_UNCHANGED)
print('Image Dimensions: ', img.shape)

Image Dimensions:  (624, 512, 3)


In [7]:
df4 = pd.read_excel('/content/drive/MyDrive/Project Data/X-ray2.xlsx')
df4.drop(df4.columns[df4.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
df4['Xray_arrays'] = 'x'

In [8]:
for i in range(0,len(df4)):
    xray1 = cv2.imread(r'/content/drive/MyDrive/Project Data/NLMCXR_png/' + df4['Image_file1'][i])
    xray1 = cv2.resize(xray1,(224,224))
    xray1 = preprocess_input(xray1)
    xray1 = tf.expand_dims(xray1, axis=0)
    feat1 = CNN1.predict(xray1, verbose=0)
    xray2 = cv2.imread(r'/content/drive/MyDrive/Project Data/NLMCXR_png/'+  df4['Image_file2'][i])
    xray2 = cv2.resize(xray2,(224,224))
    xray2 = preprocess_input(xray2)
    xray2 = tf.expand_dims(xray2, axis=0)
    feat2 = CNN1.predict(xray2, verbose=0)
    comb = np.concatenate((feat1, feat2), axis=1)
    comb = np.reshape(comb, comb.shape[1])
    df4['Xray_arrays'][i] = comb

In [9]:
df4

,Patient_ID,Image_file1,Image_file2,Report_ID,Impression,Impression_token,Xray_arrays
0,CXR1,CXR1_1_IM-0001-4001.png,CXR1_1_IM-0001-3001.png,1.xml,normal chest,BEGINnormal chestEND,"[0.00050475576, 0.0020294907, 0.0012941753, 0...."
1,CXR2,CXR2_IM-0652-1001.png,CXR2_IM-0652-2001.png,2.xml,no acute pulmonary findings,BEGINno acute pulmonary findingsEND,"[0.00024342188, 0.0017629942, 0.0013469761, 0...."
2,CXR3,CXR3_IM-1384-1001.png,CXR3_IM-1384-2001.png,3.xml,no displaced rib fractures pneumothorax or ple...,BEGINno displaced rib fractures pneumothorax o...,"[0.00044275637, 0.0014715064, 0.002141107, 0.0..."
3,CXR4,CXR4_IM-2050-2001.png,CXR4_IM-2050-1001.png,4.xml,bullous emphysema and interstitial fibrosis pr...,BEGINbullous emphysema and interstitial fibros...,"[7.3000316e-05, 0.0010577432, 0.0013300176, 0...."
4,CXR5,CXR5_IM-2117-1004003.png,CXR5_IM-2117-1003002.png,5.xml,no acute cardiopulmonary abnormality,BEGINno acute cardiopulmonary abnormalityEND,"[7.748296e-05, 0.0016451341, 0.0015252208, 0.0..."
...,...,...,...,...,...,...,...
3815,CXR3995,CXR3995_IM-2046-2001.png,CXR3995_IM-2046-1001.png,3995.xml,interval resolution of bibasilar airspace dise...,BEGINinterval resolution of bibasilar airspace...,"[2.7642482e-05, 0.0011103343, 0.0013878369, 0...."
3816,CXR3996,CXR3996_IM-2047-3001.png,CXR3996_IM-2047-1001.png,3996.xml,clear lungs no acute cardiopulmonary abnormality,BEGINclear lungs no acute cardiopulmonary abno...,"[1.8356866e-05, 0.00080837257, 0.0011780078, 0..."
3817,CXR3997,CXR3997_IM-2048-1001.png,CXR3997_IM-2048-1002.png,3997.xml,no acute findings no evidence for active tb,BEGINno acute findings no evidence for active ...,"[0.0004987752, 0.0011736292, 0.002176965, 0.0,..."
3818,CXR3998,CXR3998_IM-2048-1002.png,CXR3998_IM-2048-1001.png,3998.xml,heart size is normal and the lungs are clear,BEGINheart size is normal and the lungs are cl...,"[7.982528e-05, 0.001707057, 0.0012614432, 0.0,..."


In [10]:

df4.to_excel('/content/drive/MyDrive/Project Data/X-raywithChex.xlsx')
df4.to_pickle('/content/drive/MyDrive/Project Data/X-raywithChex.pkl') 

In [11]:
df5 = pd.read_excel('/content/drive/MyDrive/Project Data/X-raywithChex.xlsx')
df5.drop(df5.columns[df5.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

In [12]:
df5

,Patient_ID,Image_file1,Image_file2,Report_ID,Impression,Impression_token,Xray_arrays
0,CXR1,CXR1_1_IM-0001-4001.png,CXR1_1_IM-0001-3001.png,1.xml,normal chest,BEGINnormal chestEND,[5.0475576e-04 2.0294907e-03 1.2941753e-03 ......
1,CXR2,CXR2_IM-0652-1001.png,CXR2_IM-0652-2001.png,2.xml,no acute pulmonary findings,BEGINno acute pulmonary findingsEND,[2.4342188e-04 1.7629942e-03 1.3469761e-03 ......
2,CXR3,CXR3_IM-1384-1001.png,CXR3_IM-1384-2001.png,3.xml,no displaced rib fractures pneumothorax or ple...,BEGINno displaced rib fractures pneumothorax o...,[4.4275637e-04 1.4715064e-03 2.1411071e-03 ......
3,CXR4,CXR4_IM-2050-2001.png,CXR4_IM-2050-1001.png,4.xml,bullous emphysema and interstitial fibrosis pr...,BEGINbullous emphysema and interstitial fibros...,[7.3000316e-05 1.0577432e-03 1.3300176e-03 ......
4,CXR5,CXR5_IM-2117-1004003.png,CXR5_IM-2117-1003002.png,5.xml,no acute cardiopulmonary abnormality,BEGINno acute cardiopulmonary abnormalityEND,[7.7482960e-05 1.6451341e-03 1.5252208e-03 ......
...,...,...,...,...,...,...,...
3815,CXR3995,CXR3995_IM-2046-2001.png,CXR3995_IM-2046-1001.png,3995.xml,interval resolution of bibasilar airspace dise...,BEGINinterval resolution of bibasilar airspace...,[2.7642482e-05 1.1103343e-03 1.3878369e-03 ......
3816,CXR3996,CXR3996_IM-2047-3001.png,CXR3996_IM-2047-1001.png,3996.xml,clear lungs no acute cardiopulmonary abnormality,BEGINclear lungs no acute cardiopulmonary abno...,[1.8356866e-05 8.0837257e-04 1.1780078e-03 ......
3817,CXR3997,CXR3997_IM-2048-1001.png,CXR3997_IM-2048-1002.png,3997.xml,no acute findings no evidence for active tb,BEGINno acute findings no evidence for active ...,[4.9877522e-04 1.1736292e-03 2.1769649e-03 ......
3818,CXR3998,CXR3998_IM-2048-1002.png,CXR3998_IM-2048-1001.png,3998.xml,heart size is normal and the lungs are clear,BEGINheart size is normal and the lungs are cl...,[7.9825280e-05 1.7070570e-03 1.2614432e-03 ......


In [ ]:
tokens=[]
cap_len = []
for i in range (0,len(df5)):
    temp = (word_tokenize(df5['Impression_token'][i]))
    cap_len.append(len(temp))
    for j in temp:
        tokens.append(j)
vocab2 = set(tokens)vocab2 = list(vocab2)
len(vocab2)

1551

In [ ]:
list_difference = []
for item in vocab2:
  if item not in vocab:
    list_difference.append(item)

print(list_difference)

['<', 'STA', 'END', '>']


In [ ]:
print(vocab)

['mixed', 'fluid', 'rightsided', 'postinfectiouspostinflammatory', 'explained', 'mild', 'endotracheal', 'correct', 'spondylosis', 'lucent', 'dystrophy', 'oligemia', 'approximately', 'p', 'icdpacemaker', 'innominate', 'status', 'anterolisthesis', 'segments', 'reflects', 'metastatic', 'detecting', 'degenerated', 'internal', 'case', 'raises', 'flowing', 'airfilled', 'spherical', 'subpleural', 'alternatively', 'an', 'neoplastic', 'processes', 'elevation', 'lymphadenopathy', 'convexity', 'esophagogastric', 'traumatic', 'only', 'patient', 'sympathetic', 'tract', 'largest', 'expansile', 'congestion', 'decubitus', 'todays', 'otherwise', 'breast', 'impression', 'extrapleural', 'sternum', 'nearcomplete', 'institution', 'thorax', 'greater', 'cholelithiasis', 'intraabdominal', 'dextroscoliosis', 'replacement', 'given', 'fragments', 'postsurgical', 'malignancy', 'fibrotic', 'hand', 'hodgkins', 'vasculature', 'endplate', 'poorly', 'medially', 'posterior', 'improvement', 'subsequent', 'sternoclavicul

In [ ]:
cap_len = sorted(cap_len)
x = np.quantile(cap_len, [0,0.25,0.5,0.75,0.80,0.95,0.99,1])
print('The minimum length of a caption is :',min(cap_len))
print('The maximum length of a caption is :',max(cap_len))
print('The 25th percentile length of a caption is :',x[1])
print('The 50th percentile length of a caption is :',x[2])
print('The 75th percentile length of a caption is :',x[3])
print('The 80th percentile length of a caption is :',x[4])
print('The 95th percentile length of a caption is :',x[5])
print('The 99th percentile length of a caption is :',x[6])

The minimum length of a caption is : 1
The maximum length of a caption is : 122
The 25th percentile length of a caption is : 4.0
The 50th percentile length of a caption is : 5.0
The 75th percentile length of a caption is : 11.0
The 80th percentile length of a caption is : 14.0
The 95th percentile length of a caption is : 33.0
The 99th percentile length of a caption is : 53.809999999999945


In [ ]:
sum(i < 54 for i in cap_len)

3781

In [ ]:
train, test = train_test_split(df5, test_size=0.2, random_state=75, shuffle=True)

In [ ]:
train.shape

(3056, 7)

In [ ]:
test.shape

(764, 7)

In [ ]:
df5['Impression_token'][7]

'<STA> no acute cardiopulmonary disease <END>'

In [ ]:
# Havent run yet...check and then run #
tokenizer = Tokenizer()
impression_tokens = []
for i in range(0,len(df5)):
    impression_tokens.append(df5['Impression_token'][i])
tokenizer.fit_on_texts(impression_tokens)
seq = tokenizer.texts_to_sequences(['<STA> interval improvement in consolidative left base opacity multifocal scattered bibasilar patchy and pulmonary opacities again noted most consistent with atelectasisinfiltrate stable enlarged cardiomediastinal silhouette stable pulmonary vascular congestion <END>'])[0]

In [ ]:
for i in range(1, len(seq)):
                    # split into input and output pairs
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=54)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=1547)[0]
                    
                    # store the sequences
                    X1.append(features[key][0])
                    X2.append(in_seq)
                    y.append(out_seq)

NameError: name 'X1' is not defined

In [ ]:
in_seq, out_seq = seq[0:5], seq[5:6]

In [ ]:
in_seq = pad_sequences([in_seq], maxlen=74, padding='post' ,truncating='post')[0]

In [ ]:
out_seq = to_categorical([out_seq], num_classes=1547)[0]

0.0

In [ ]:
encoder = OneHotEncoder()
arr= encoder.fit_transform([out_seq]).toarray()

In [ ]:
len(tokenizer.word_index)

1549

In [ ]:
arr

array([[1.]])

In [ ]:
count = out_seq[out_seq==0].shape[0]
count

1546

In [ ]:
impression_tokens

['<STA> normal chest <END>',
 '<STA> no acute pulmonary findings <END>',
 '<STA> no displaced rib fractures pneumothorax or pleural effusion identified wellexpanded and clear lungs mediastinal contour within normal limits no acute cardiopulmonary abnormality identified <END>',
 '<STA> bullous emphysema and interstitial fibrosis probably scarring in the left apex although difficult to exclude a cavitary lesion opacities in the bilateral upper lobes could represent scarring however the absence of comparison exam recommend short interval followup radiograph or ct thorax to document resolution <END>',
 '<STA> no acute cardiopulmonary abnormality <END>',
 '<STA> no acute cardiopulmonary findings <END>',
 '<STA> basilar atelectasis no confluent lobar consolidation or pleural effusion <END>',
 '<STA> no acute cardiopulmonary disease <END>',
 '<STA> increased size of density in the left cardiophrenic primary differential considerations include increased size of prominent epicardial fat pericar

In [ ]:
seq

[1, 2, 3, 4, 5, 6]

In [ ]:
twt = ['He is a lazy person.']
twt = tokenizer.texts_to_sequences(twt)[0]
print (twt)

[]


In [ ]:
a = [[1, 2, 3], [3, 4, 5, 6], [7, 8]]

In [ ]:
pad_sequences(a)

array([[0, 1, 2, 3],
       [3, 4, 5, 6],
       [0, 0, 7, 8]])

In [ ]:
pad_sequences(a, padding="post")

array([[1, 2, 3, 0],
       [3, 4, 5, 6],
       [7, 8, 0, 0]])